## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Punta Arenas,-53.1500,-70.9167,44.71,76,40,39.12,CL,scattered clouds
1,1,Puerto Escondido,15.8500,-97.0667,81.43,74,1,5.75,MX,clear sky
2,2,Kodiak,57.7900,-152.4072,26.11,63,90,11.50,US,overcast clouds
3,3,Ponta Do Sol,32.6667,-17.1000,63.72,64,3,5.30,PT,clear sky
4,4,Upernavik,72.7868,-56.1549,7.43,65,34,8.61,GL,scattered clouds


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
1,1,Puerto Escondido,15.8500,-97.0667,81.43,74,1,5.75,MX,clear sky
7,7,Vaini,-21.2000,-175.2000,84.36,79,20,9.22,TO,few clouds
16,16,Hadejia,12.4498,10.0444,75.00,18,8,10.67,NG,clear sky
17,17,Hambantota,6.1241,81.1185,80.56,78,71,15.17,LK,broken clouds
21,21,Beira,-19.8436,34.8389,76.84,86,100,8.48,MZ,light rain
23,23,Hithadhoo,-0.6000,73.0833,82.38,76,94,21.61,MV,overcast clouds
27,27,Canton,23.1167,113.2500,80.01,62,96,7.45,CN,overcast clouds
33,33,Caucaia,-3.7361,-38.6531,82.35,72,20,12.66,BR,few clouds
34,34,Kupang,-10.1667,123.5833,88.38,70,20,11.50,ID,few clouds
37,37,Ternate,0.8000,127.4000,86.20,65,38,1.59,ID,scattered clouds


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                174
City                   174
Lat                    174
Lng                    174
Max Temp               174
Humidity               174
Cloudiness             174
Wind Speed             174
Country                174
Current Description    174
dtype: int64

In [24]:
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
1,1,Puerto Escondido,15.8500,-97.0667,81.43,74,1,5.75,MX,clear sky
7,7,Vaini,-21.2000,-175.2000,84.36,79,20,9.22,TO,few clouds
16,16,Hadejia,12.4498,10.0444,75.00,18,8,10.67,NG,clear sky
17,17,Hambantota,6.1241,81.1185,80.56,78,71,15.17,LK,broken clouds
21,21,Beira,-19.8436,34.8389,76.84,86,100,8.48,MZ,light rain


In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_hotel_df= preferred_cities_df.dropna()

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Escondido,MX,81.43,clear sky,15.8500,-97.0667,
7,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,
16,Hadejia,NG,75.00,clear sky,12.4498,10.0444,
17,Hambantota,LK,80.56,broken clouds,6.1241,81.1185,
21,Beira,MZ,76.84,light rain,-19.8436,34.8389,
23,Hithadhoo,MV,82.38,overcast clouds,-0.6000,73.0833,
27,Canton,CN,80.01,overcast clouds,23.1167,113.2500,
33,Caucaia,BR,82.35,few clouds,-3.7361,-38.6531,
34,Kupang,ID,88.38,few clouds,-10.1667,123.5833,
37,Ternate,ID,86.20,scattered clouds,0.8000,127.4000,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df= hotel_df.dropna(subset = ["Hotel Name"], inplace=False)

In [30]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Name</dt><dd>{City}</dd>
<dt>Another name</dt><dd>{Country}</dd>
<dt>weather description</dt><dd>{Current Description}</dd>
<dt>Maximum temperature </dt><dd>{Max Temp}</dd>
</dl> """

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))